Original code from capstone_recommendation.py

In [ ]:
connection = pymysql.connect(host='35.226.35.190',
                             user='root',
                             password='capstone',
                             db='capstone')

query = '''SELECT * FROM QA_FINAL'''

data = pd.read_sql(query, connection)



In [151]:
import pandas as pd
import json
import pymysql
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re, string

In [2]:
data = pd.read_csv('../../data-science-for-good-careervillage/data/qa_dataset_final.csv')

In [22]:
data.head()

,answers_id,answers_body_clean,answers_score,questions_id,questions_body_clean,questions_score
0,4e5f01128cae4f6d8fd697cec5dca60c,Hi! You are asking a very interesting question...,0.0,332a511f1569444485cf7a7a556a5e54,What is a maths teacher? what is a ma...,1.0
1,ada720538c014e9b8a6dceed09385ee3,Hi. I joined the Army after I attended college...,0.0,eb80205482e4424cad8f16bc25aa2d9c,I am Priyanka from Bangalore . Now am in 10th ...,5.0
2,eaa66ef919bc408ab5296237440e323f,"Dear Priyanka, Greetings! I have answered this...",0.0,eb80205482e4424cad8f16bc25aa2d9c,I am Priyanka from Bangalore . Now am in 10th ...,5.0
3,1a6b3749d391486c9e371fbd1e605014,I work for a global company who values highly ...,0.0,4ec31632938a40b98909416bdd0decff,I'm planning on going abroad for my first job....,2.0
4,5229c514000446d582050f89ebd4e184,I agree with Denise. Every single job I've had...,0.0,2f6a9a99d9b24e5baa50d40d0ba50a75,i hear business management is a hard way to ge...,2.0


In [160]:
occ_df.head()

,Career Cluster,Career Pathway,Code,Occupation
0,"Agriculture, Food & Natural Resources",Agribusiness Systems,11-9013.03,Aquacultural Managers
1,"Agriculture, Food & Natural Resources",Agribusiness Systems,13-1021.00,"Buyers and Purchasing Agents, Farm Products"
2,"Agriculture, Food & Natural Resources",Agribusiness Systems,11-9013.02,Farm and Ranch Managers
3,"Agriculture, Food & Natural Resources",Agribusiness Systems,13-1074.00,Farm Labor Contractors
4,"Agriculture, Food & Natural Resources",Agribusiness Systems,11-9013.00,"Farmers, Ranchers, and Other Agricultural Mana..."


In [189]:
occ_df.loc[occ_df['Occupation'] == 'Data Warehousing Specialists','Code'].iloc[0]

'15-1199.07'

In [192]:
import string

# read in BLS career dataset
occ_df = pd.read_csv('../0_data/All_Career_Clusters.csv')

# get just the occupation names
occupations = soc_data['Occupation'].to_list()

# split up question dataset
questions_ids = list(data.questions_id.unique())
questions_df = data[data.questions_id.isin(questions_ids)].dropna()
questions = list(questions_df.questions_body_clean.unique())


vec = TfidfVectorizer(max_features=5000)
vec.fit_transform(questions)

# Vectorize the question and occupation lists
vec_matrix = vec.transform(questions).todense()
occ_matrix = vec.transform([occ.lower() for occ in occupations]).todense()

def find_answers(q_id):
    answers = list(data[data.questions_id == q_id].answers_body_clean)
    return answers

def find_similar_questions(question_body):
    
    # remove punctuation from input question and make lower-case
    question_body_clean = question_body.lower().translate(str.maketrans('', '', string.punctuation))
    
    # vectorize input question
    x = vec.transform([question_body_clean]).todense()
    
    # first get the similar questions
    scores = cosine_similarity(x, vec_matrix)
    similar_inds = scores.argsort()[0][-6:-1]
    similar_qs = pd.DataFrame([questions[i] for i in similar_inds], columns = {'questions_body_clean'})
    similar_q_join = pd.merge(left = similar_qs, right = questions_df, on ='questions_body_clean', how='inner')
    out = similar_q_join[['questions_id','questions_body_clean','answers_score','answers_body_clean']]
    
    # now find the most similar BLS occupation name
    occ_scores = cosine_similarity(x, occ_matrix)
    best_ind = occ_scores.argsort()[0][-1]
    best_occ = occupations[best_ind]
    # get the corresponding SOC code for that job
    occ_soc = occ_df.loc[occ_df['Occupation'] == best_occ,'Code'].iloc[0]
        
    # add the input question, closest occupation and occupation SOC code to the output
    out['input_question_body']=question_body
    out['closest_occupation'] = best_occ
    out['soc_code'] = str(occ_soc)
    
    # rename some columns
    out = out.rename(columns={'questions_id':'question_id','questions_body_clean':'question_body','answers_score':'score',
                        'answers_body_clean':'answers'})
    
    return out
    

In [193]:
find_similar_questions('How DO I BECOME an financial advisor?')

/Users/lucyxie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,question_id,question_body,score,answers,input_question_body,closest_occupation,soc_code
0,7838db94f0bd4c7e99c8f1663276ec70,#financial-planning,0.0,Hello Ashley: Here are a few things you could ...,How DO I BECOME an financial advisor?,Financial Quantitative Analysts,13-2099.01
1,cf59ffeeff8047518169110ef16821e4,I am thinking of pursuing a degree in finance ...,0.0,"Terrie - I agree with Sonali Agrawal. However,...",How DO I BECOME an financial advisor?,Financial Quantitative Analysts,13-2099.01
2,cf59ffeeff8047518169110ef16821e4,I am thinking of pursuing a degree in finance ...,0.0,Personal Financial advisor is a good choice. A...,How DO I BECOME an financial advisor?,Financial Quantitative Analysts,13-2099.01
3,2125061f59fa4d73826989f2bd85030f,To help me decide which major to take in colle...,2.0,Dear MY.N\nWow those are both huge goals. I wo...,How DO I BECOME an financial advisor?,Financial Quantitative Analysts,13-2099.01
4,2125061f59fa4d73826989f2bd85030f,To help me decide which major to take in colle...,2.0,What does a Nurse Anesthesia Education Program...,How DO I BECOME an financial advisor?,Financial Quantitative Analysts,13-2099.01
5,856b62dfdfa447d7ab26d3bc0371e6b7,I want to become a financial advisor after col...,0.0,"Hi Mikayla,\n\nIf you chose Economics or Finan...",How DO I BECOME an financial advisor?,Financial Quantitative Analysts,13-2099.01
6,b604da1702ca4c69a39aba0805c3e4c7,I've been interested in becoming a financial a...,1.0,"Hi Tiffany, I majored in Finance in college t...",How DO I BECOME an financial advisor?,Financial Quantitative Analysts,13-2099.01
7,b604da1702ca4c69a39aba0805c3e4c7,I've been interested in becoming a financial a...,0.0,Hi Tiffany! Thank you so much for asking this...,How DO I BECOME an financial advisor?,Financial Quantitative Analysts,13-2099.01
8,b604da1702ca4c69a39aba0805c3e4c7,I've been interested in becoming a financial a...,0.0,I would suggest following your interests withi...,How DO I BECOME an financial advisor?,Financial Quantitative Analysts,13-2099.01
9,b604da1702ca4c69a39aba0805c3e4c7,I've been interested in becoming a financial a...,0.0,Hi - I support several of the comments posted ...,How DO I BECOME an financial advisor?,Financial Quantitative Analysts,13-2099.01


In [115]:
df = pd.DataFrame(columns=['user_question','question_body','answers'])
for q in test_questions:
    test = find_similar_questions(q)
    test_combined = pd.DataFrame(test.groupby('question_body')['answers'].apply(lambda x: "%s" % ', '.join(x))).reset_index()
    test_combined['user_question']=q    
    df = df.append(test_combined, sort=False)

In [116]:
df.head()

,user_question,question_body,answers
0,What does a data scientist do?,"Hello, I am going to a liberal art college ...","Hi Yingyi, Great question and excited to see ..."
1,What does a data scientist do?,I want to be a scientist #scientist #researcher,Researcher is a very thrilling and demanding p...
2,What does a data scientist do?,I'm currently a third year computer science ma...,"Hey Albert, awesome to see you're interested i..."
3,What does a data scientist do?,"I've been looking into data science careers, a...",Data Science is a rather broad field that cove...
4,What does a data scientist do?,What kind of companies do data scientists work...,"The definition of this newfangled title ""data ..."
